# 1 
The 4 Bellman Policy Equations

For a deterministic policy, MDP --> MRP ?


We choose action $a$ with probability 1: $a=\pi(s)$; this action only depends on the state $s$, so I drop it from an input to $Q$ and $V$. We basically end up with the MRP equations

$V^{\pi_D}(s) = Q^{\pi_D}(s)$ 

$Q^{\pi_D}(s) = R(s) + \gamma  \sum_{s' \in N} P(s,s')  V^{\pi_D}(s')$

$V^{\pi_D}(s) = R^\pi(s) + \gamma \sum_{s' \in N} P(s,s')  V^{\pi_D}(s')$

$Q^{\pi_D}(s) = R(s) + \gamma  \sum_{s' \in N} P(s,s')  Q^{\pi_D}(s')$

# 2

$V_*(s) = max_{\pi \in \prod} V^\pi(s) = max_a Q_*(s,a)$

$R_s(a) = a(1-a) + (1-a)(1+a) = a - a^2 + 1 - a^2 = -2a^2 + a + 1$

$\frac{\partial R}{\partial a} = -4a + 1$, equals $0$ when $a=0.25$

$\max R_s(a) = R_s(0.25) = 1.125$


Discount across all future states. We will choose $a=0.25$ at every step, which gives us an expected reward of 1.125 each time. Hence, we need to discount 1.125 indefinitely into the future.

Terminal value = $\frac{1.125}{0.50} = 2.25$

Optimal Deterministic Policy: $\pi^*(s) = 0.25$

# 3